<a href="https://colab.research.google.com/github/Asritha0606/GenAIcohort_May2025/blob/main/AI_Recruiter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task


Create a Multiagent system as an AI Recruiter using crew ai that takes sample resumes and an HR query as input and returns the most matching resumes.

Setup and dependencies

In [ ]:
!pip install -q crewai 'crewai[tools]' --progress-bar off together

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.52.4 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

# hf_api_key = userdata.get('huggingfacehub')
# os.environ["HUGGINGFACEHUB_API_KEY"] = hf_api_key

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

gemini_api_key = userdata.get('Gemini_api_2')
os.environ["GEMINI_API_KEY"] = gemini_api_key

together_api_key = userdata.get('together_api')
os.environ["TOGETHER_API_KEY"] = together_api_key

In [ ]:
from crewai import LLM
llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)

Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [ ]:
from crewai import Agent, Task, Crew
# from langchain_groq.chat_models import ChatGroq # Corrected import
import os

In [ ]:
!pip install -q crewai 'crewai[tools]' --progress-bar off together
!pip install -q langchain groq langchain-groq langchain-community crewai_tools

from crewai import Agent, Task, Crew, LLM
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)


try:
    search_tool = SerperDevTool(api_key=serper_api_key)
    print("Serper Dev Tool instantiated successfully.")
except Exception as e:
    print(f"Failed to instantiate Serper Dev Tool: {e}")

try:
    scrape_tool = ScrapeWebsiteTool()
    print("Scrape Website Tool instantiated successfully.")
except Exception as e:
    print(f"Failed to instantiate Scrape Website Tool: {e}")

try:
    # Assuming the file exists from previous steps or will be created later
    read_resume = FileReadTool(file_path='/content/Asritha_Sai_Resume.md')
    print("File Read Tool instantiated successfully.")
except Exception as e:
    print(f"Failed to instantiate File Read Tool: {e}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.2 MB/s eta 0:00:00
Serper Dev Tool instantiated successfully.
Scrape Website Tool instantiated successfully.
File Read Tool instantiated successfully.


In [ ]:
#read_resume = FileReadTool()

Agents

In [ ]:
resume_reader = Agent(
    role='Resume Reader',
    goal='Read and extract the raw text content from various resume files.',
    backstory="""As a dedicated Resume Reader, you are meticulous in your task of accessing
    and extracting the complete textual information from resume documents.
    Your primary function is to provide the raw data for analysis.""",
    tools=[read_resume],
    llm=llm,
    verbose=False,
    allow_delegation=False
)

resume_analyzer = Agent(
    role='Resume Analyzer',
    goal='Analyze resume content to extract key skills, experience, and qualifications relevant to a given HR query.',
    backstory="""Equipped with a keen eye for detail, you are the Resume Analyzer.
    You process the raw resume text, identify and categorize critical information
    like technical skills, work history, education, and achievements,
    specifically focusing on criteria provided in the HR query.""",
    tools=[search_tool],
    llm=llm,
    verbose=False,
    allow_delegation=True
)

hiring_manager = Agent(
    role='Hiring Manager',
    goal='Compare analyzed resumes against the HR query and select the most matching candidates.',
    backstory="""As the final decision-maker, you are the Hiring Manager.
    You take the analyzed resume data and the HR query, evaluate the fit,
    and determine which candidates are the most suitable for the role.""",
    tools=[search_tool],
    llm=llm,
    verbose=False,
    allow_delegation=False
)

print("Agents defined and instantiated successfully.")

Agents defined and instantiated successfully.


Tasks

In [ ]:
read_resume_task = Task(
    description=(
        "Read the content of the resume file located at {resume_file_path}."
        "Extract the complete raw text from the file."
    ),
    expected_output="A string containing the full text content of the resume file.",
    agent=resume_reader,
    tools=[read_resume]
)

analyze_resume_task = Task(
    description=(
        "Analyze the following resume text:\n\n"
        "{resume_text}\n\n"
        "Based on the HR query: '{hr_query}', extract and summarize the candidate's "
        "key skills, work experience, education, and qualifications that are directly "
        "relevant to the query. Identify strengths and weaknesses relative to the query."
    ),
    expected_output=(
        "A structured summary of the resume highlighting relevant skills, experience, "
        "education, and qualifications based on the HR query. "
        "Include a brief assessment of the candidate's fit for the role."
    ),
    agent=resume_analyzer,
    context=[read_resume_task],
    tools=[search_tool]
)

select_candidates_task = Task(
    description=(
        "Compare the analyses of the following resumes against the HR query: '{hr_query}'.\n\n"
        "Resume Analyses:\n{resume_analyses}\n\n"
        "Identify the top candidates based on the relevance of their skills, "
        "experience, and qualifications as summarized in the analyses, compared to the HR query. "
        "Provide a ranked list of the most matching candidates with a brief justification for each."
    ),
    expected_output=(
        "A ranked list of the most suitable candidates from the provided analyses, "
        "including their name or resume identifier and a brief justification for their ranking "
        "based on the HR query and the analysis."
    ),
    agent=hiring_manager,
    context=[analyze_resume_task],
    tools=[search_tool]
)

print("Tasks defined successfully.")

Tasks defined successfully.


Tasks for each resume and starting of Crew


In [ ]:
# 1. Define the input HR query
hr_query = "We are looking for a data scientist with strong Python and machine learning skills, experience with cloud platforms like AWS or GCP, and a background in statistical modeling."

# 2. Define the list of resume file paths

resume_files = ["/content/data/resume1.txt", "/content/data/resume2.txt", "/content/data/resume3.txt"]



In [ ]:

read_tasks = [Task(
    description=(
        "Read the content of the resume file located at {{ resume_file_path }}."
        "Extract the complete raw text from the file."
    ),
    expected_output=f"A string containing the full text content of the resume file {{ resume_file_path }}.",
    agent=resume_reader,
    tools=[read_resume],
    # Pass the file path as an input parameter to the task
    inputs={"resume_file_path": file_path},
    task_id=f"read_{i}" # Add a unique ID for context
) for i, file_path in enumerate(resume_files)]

analyze_tasks = [Task(
    description=(
        f"Analyze the following resume text from {resume_files[i]}:\n\n"
        "{{ read_" + str(i) + ".output }}\n\n" # Use Jinja2 syntax to reference the output of the corresponding read task by task_id and '.output'
        f"Based on the HR query: '{hr_query}', extract and summarize the candidate's "
        "key skills, work experience, education, and qualifications that are directly "
        "relevant to the query. Identify strengths and weaknesses relative to the query."
        f"Label this analysis clearly as 'Analysis for {resume_files[i]}'."
    ),
    expected_output=(
        f"A structured summary of the resume from {resume_files[i]} highlighting relevant skills, experience, "
        "education, and qualifications based on the HR query. "
        "Include a brief assessment of the candidate's fit for the role."
    ),
    agent=resume_analyzer,
    context=[read_tasks[i]], # Depends on the corresponding read task
    tools=[search_tool],
    task_id=f"analyze_{i}" # Add a unique ID for context
) for i in range(len(resume_files))]

# The final selection task depends on ALL analyze tasks
select_candidates_task.context = analyze_tasks
select_candidates_task.description = (
    f"Compare the analyses of the following {len(resume_files)} resumes against the HR query: '{hr_query}'.\n\n"
    "Resume Analyses:\n"
    # Use Jinja2 syntax to collect output from all analyze tasks in the context
    + "\n".join([f"{{{{ analyze_" + str(i) + ".output }}}}" for i in range(len(resume_files))]) +
    "\n\n"
    "Identify the top candidates based on the relevance of their skills, "
    "experience, and qualifications as summarized in the analyses, compared to the HR query. "
    "Provide a ranked list of the most matching candidates with a brief justification for each."
    "Output format should be a clear list, e.g., '1. Candidate A (Justification), 2. Candidate B (Justification)...'"
)


# Combine all tasks into a single list for the crew, maintaining order for sequential execution
all_tasks = read_tasks + analyze_tasks + [select_candidates_task]


# 3. Instantiate the Crew
recruiter_crew = Crew(
    agents=[resume_reader, resume_analyzer, hiring_manager],
    tasks=all_tasks,
    verbose=False # Fix: Set verbose to True
)

# 4. Execute the crew
crew_result = recruiter_crew.kickoff(inputs={'hr_query': hr_query})


print("\n## Crew Execution Complete ##")
print("Crew Result:")
print(crew_result)


## Crew Execution Complete ##
Crew Result:
1. Candidate from resume2.txt (Strong Python skills, multiple ML models, strong academic background, and relevant projects. While lacking explicit cloud platform experience, their AI/ML focused internships and projects demonstrate practical application and potential. Further questioning during the interview can assess their statistical modeling experience and cloud platform knowledge.)  2. Candidate from resume3.txt (Similar strong Python and ML skills to candidate 2, with experience in diverse ML subfields. However, also lacks explicit cloud platform experience and detailed statistical modeling background.  Further probing during the interview will be necessary.) 3.  Candidate from resume1.txt (No information provided for this candidate, hence no ranking possible.)


In [ ]:
from IPython.display import Markdown
Markdown(str(crew_result))

1. Candidate from resume2.txt (Strong Python skills, multiple ML models, strong academic background, and relevant projects. While lacking explicit cloud platform experience, their AI/ML focused internships and projects demonstrate practical application and potential. Further questioning during the interview can assess their statistical modeling experience and cloud platform knowledge.)  2. Candidate from resume3.txt (Similar strong Python and ML skills to candidate 2, with experience in diverse ML subfields. However, also lacks explicit cloud platform experience and detailed statistical modeling background.  Further probing during the interview will be necessary.) 3.  Candidate from resume1.txt (No information provided for this candidate, hence no ranking possible.)

Gradio


In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def run_ai_recruiter(hr_query, files):
    read_tasks, analyze_tasks = [], []

    # Dynamic resume read + analyze tasks
    for i, file_obj in enumerate(files):
        resume_path = file_obj.name
        task_id = f"resume_{i}"

        read_task = Task(
            description=f"Extract resume content from: {resume_path}",
            expected_output="Full resume text",
            agent=resume_reader,
            tools=[read_resume],
            inputs={"resume_file_path": resume_path},
            task_id=f"read_{i}"
        )

        analyze_task = Task(
            description=(
                f"Analyze resume for: {hr_query}\n\n"
                f"{{{{ read_{i}.output }}}}"
            ),
            expected_output="Structured analysis of the resume",
            agent=resume_analyzer,
            tools=[search_tool],
            context=[read_task],
            task_id=f"analyze_{i}"
        )

        read_tasks.append(read_task)
        analyze_tasks.append(analyze_task)

    # Final task: Hiring manager selects best
    select_task = Task(
        description=(
            f"Compare resumes for: {hr_query}\n\n" +
            "\n".join([f"{{{{ analyze_{i}.output }}}}" for i in range(len(files))])
        ),
        expected_output="Ranked candidate list with justifications.",
        agent=hiring_manager,
        tools=[search_tool],
        context=analyze_tasks
    )

    all_tasks = read_tasks + analyze_tasks + [select_task]

    crew = Crew(
        agents=[resume_reader, resume_analyzer, hiring_manager],
        tasks=all_tasks,
        verbose=False
    )

    result = crew.kickoff(inputs={"hr_query": hr_query})
    return result

In [ ]:
demo = gr.Interface(
    fn=run_ai_recruiter,
    inputs=[
        gr.Textbox(label="HR Query", placeholder="Looking for a Python + AWS data scientist"),
        gr.File(label="Upload Resume .txt files", file_types=['.txt'], file_count="multiple")
    ],
    outputs=gr.Textbox(label="Matched Candidates"),
    title="🧠 AI Recruiter",
    description="Multi-Agent Resume Matcher with CrewAI"
)

demo.launch(debug=True)